In [ ]:
pip install JPype1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras import regularizers

from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.models import model_from_json
from keras.models import load_model
from wordcloud import WordCloud, STOPWORDS
import pylab as pl
import jpype as jp
from pathlib import Path
import os
import nltk
import re
from nltk.corpus import stopwords
from tqdm import tqdm

In [ ]:
ZEMBEREK_PATH = r'../input/zemberek/zemberek-full.jar'
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

In [ ]:
TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')
TurkishSpellChecker = jp.JClass('zemberek.normalization.TurkishSpellChecker')
TurkishSentenceNormalizer = jp.JClass('zemberek.normalization.TurkishSentenceNormalizer')
Paths = jp.JClass('java.nio.file.Paths')
# Get the path to the (baseline) lookup files
lookupRoot = Paths.get(r'../input/zemberek2/normalization')
# Get the path to the compressed bi-gram language model
lmPath = Paths.get(r'../input/zemberek3/lm.2gram.slm')
morphology = TurkishMorphology.createWithDefaults()
# Initialize the TurkishSentenceNormalizer class
# Instantiate the morphology class with the default RootLexicon
morph = TurkishMorphology.createWithDefaults()

# Instantiate the spell checker class using the morphology instance
spell = TurkishSpellChecker(morph)

#normalizer = TurkishSentenceNormalizer(morphology, lookupRoot, lmPath)

In [ ]:
def reverse(s): 
    if len(s) == 0: 
        return s 
    else: 
        return reverse(s[1:]) + s[0] 

def checkOpennes(word):
    vowels=['a','e','i','ı','o','ö','u','ü']
    open_vowels=['e','i','ü','ö']
    close_vowels=['a','ı','o','u']
    for i in range(len(word)):
        if reverse(word)[i] in vowels:
            if reverse(word)[i] in open_vowels:
                return True
            else:
                return False
        else:
            continue
def PresentCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if reverse(word)[i] in ei:
                return 'ei'
            elif reverse(word)[i] in üö:
                return 'üö'
            elif reverse(word)[i] in aı:
                return 'aı'
            elif reverse(word)[i] in uo:
                return 'uo'
            else:
                continue
    
def StartCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if (word)[i] in ei:
                return 'ei'
            elif (word)[i] in üö:
                return 'üö'
            elif (word)[i] in aı:
                return 'aı'
            elif (word)[i] in uo:
                return 'uo'
            else:
                continue

In [ ]:
WORDS = dict()

spell_folder = Path("../input/turkish-spell-check")

def words(text): return re.findall(r'\w+', text.lower())
with open(os.path.expanduser(Path(spell_folder/ "big2.txt")), "r", encoding = 'utf-8') as f:
    for line in f:
        splitted = line.split()
        WORDS[splitted[0]] = int(splitted[1])

In [ ]:
def replaceall(s, n,a):
    occurence = s.count(n)
    alt = []
    temp = s
    for i in range(occurence):
        temp2 = temp
        for j in range(i,occurence):
            temp2 = temp2.replace(n,a,1)
            alt.append(temp2)
        temp = temp.replace(n,"!",1)
    for i in range(len(alt)):
        alt[i] = alt[i].replace("!",n)

    return alt

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    if  word in WORDS.keys():
        number = WORDS[word]
    else:
        number = 1
    if number == 0:
        number = 1
    return number / N

def correction(word):
      return max(candi(word), key=P)

def candi(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcçdefgğhıijklmnoöprsştuüvyzw'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
 
    sp     = replaceall(word,'ı','i')
    sp2     = replaceall(word,'u','ü')
    sp3    = replaceall(word,'o','ö')
    sp4     = replaceall(word,'g','ğ')
    sp5     = replaceall(word,'c','ç')
    sp6     = replaceall(word,'s','ş')
    sp7     = replaceall(word,'i','ı')
    sp8     = replaceall(word,'ö','o')
    sp9     = replaceall(word,'ş','s')
    sp10     = replaceall(word,'ğ','g')
    sp11     = replaceall(word,'ç','c')
    sp12     = replaceall(word,'ü','u')
    specials=[]
    specials.extend(sp)
    specials.extend(sp2)
    specials.extend(sp3)
    specials.extend(sp4)
    specials.extend(sp5)
    specials.extend(sp6)
    specials.extend(sp7)
    specials.extend(sp8)
    specials.extend(sp9)
    specials.extend(sp10)
    specials.extend(sp11)
    specials.extend(sp12)
    return set(deletes+transposes+replaces+inserts+specials)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def print_diff(word, s):
    if not word == s:
        print(word + " --> " + s)
counter = 0


In [ ]:
morphology = TurkishMorphology.createWithDefaults()

def lemmatizer(word,texts):
        wordList=[]
        wordList = re.sub("[^\w]", " ",  texts).split()
        if '�' in word:
            return 'question'
        pos=wordList.index(word)
        if word=="SMILEYPOSITIVE":
            return word
        if word=="SMILEYNEGATIVE":
            return word
        sakin=''
        word=correction(word)
        if len(wordList)-pos>3 and pos>2:
            for i, kelime in enumerate(wordList[pos-3:pos+4]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos>5:
            for i, kelime in enumerate(wordList[pos:pos+5]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos<=5:
            for i, kelime in enumerate(wordList[pos:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)-pos<1 and pos>3:
            for i, kelime in enumerate(wordList[pos-3:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)<3:
            for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        else:
             for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        results = morphology.analyze(word)
        lemma=[]
        form=[]
        l=[]
        m=[]
        for i, result in enumerate(results):
            form.append(str(result.formatLong()))
            lemma.append(result.getLemmas()[0])
        if len(lemma)>1:
                analysis = morphology.analyzeSentence(sakin)
                results = morphology.disambiguate(sakin, analysis).bestAnalysis()
                for i, result in enumerate(results):
                        l.append(result.getLemmas()[0])
                        m.append(result.formatLong())
                for i in range(len(m)):
                    for j in range(len(form)):
                        if m[i]==form[j]:
                            lema=lemma[j]
                            if lema=='değil':
                                return 'değil'
                            if 'Neg' in form[j] or 'WithoutHavingDoneSo' in form[j] or 'Unable' in form[j]:
                                if checkOpennes(word):
                                    return lema+'me'
                                else:
                                    return lema+'ma'
                            if 'Without' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'siz'
                                elif PresentCheck(word)=='aı':
                                    return lema+'sız'
                                elif PresentCheck(word)=='uo':
                                    return lema+'suz'
                                else:
                                    return lema+'süz'
                            if 'With' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'li'
                                elif PresentCheck(word)=='aı':
                                    return lema+'lı'
                                elif PresentCheck(word)=='uo':
                                    return lema+'lu'
                                else:
                                    return lema+'lü'
                            else:
                                return lema
                    else:
                        continue
        elif len(lemma)==1:
            if lemma[0]=='değil':
                return lemma[0]
            if 'Neg' in form[0] or 'WithoutHavingDoneSo' in form[0] or 'Unable' in form[0]:
                 if checkOpennes(word):
                    return lemma[0]+'me'
                 else:
                    return lemma[0]+'ma'
            elif 'Without' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'siz'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'sız'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'suz'
                else:
                    return lemma[0]+'süz'
            elif 'With' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'li'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'lı'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'lu'
                else:
                    return lemma[0]+'lü'
            else:
                return lemma[0]
        else:
            return word

In [ ]:
#from snowballstemmer import TurkishStemmer
#turkStem=TurkishStemmer()
#turkStem.stemWord("") #ekmek
#word = "ses kalitesi ve ergonomisi rezalet sony olduğu için aldım ama de fiyatına çin replika ürün alsaydım çok çok daha iyiydi kesinlikle tavsiye etmiyorum"
#word = word.split()
#word = [turkStem.stemWord(p) for p in word]
#word = " ".join(word)
#print(word)

In [ ]:
dataset_train = pd.read_csv("../input/duygu-analizi-icin-urun-yorumlari/magaza_yorumlari_duygu_analizi.csv", encoding="utf-16")
sentiment = {'Tarafsız': 0,'Olumsuz': -1, "Olumlu": 1}
dataset_train.Durum = [sentiment[item] for item in dataset_train.Durum]
dataset_train = dataset_train.dropna()
print(dataset_train.Görüş[0])

In [ ]:
description_list = []
stop_words = stopwords.words('turkish')
for description in tqdm(dataset_train.Görüş):
    #Regular Expression
    description = re.sub("[^a-zA-ZığçşüöİĞÜÇÖŞ]", " ", description)
    description = description.replace('I','ı')
    description = description.replace('İ','i') 
    description = description.lower()
    description = nltk.word_tokenize(description)
    metin = " ".join(description)
    for i, word in enumerate(description):
        description[i] = lemmatizer(word,metin)
    description = [str(word) for word in description if not word in stopwords.words()]
    description = " ".join(description)
    description_list.append(description)

In [ ]:
dataset_train = pd.read_csv("../input/lemmatizeson/lemmatize_son.csv")
X = dataset_train.text.values
y = dataset_train.sentiment.values

In [ ]:
# sıfırdan yapıldığında bura
X = np.array(description_list)
y = dataset_train.Durum.values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

y_train = pd.Series(y_train)
y_test = pd.Series(y_test)

#dico = {"text":X, "sentiment":y}
#dfo = pd.DataFrame(data=dico)
#dfo.to_csv ("dataset_train.csv", index = False, header=True)


from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
y_train = to_categorical(y_train, num_classes = 3)
y_test = to_categorical(y_test, num_classes = 3)
print(X_train)
print(y_train)

In [ ]:
dataset_train = pd.read_csv("../input/trke-lemmatized/dataset_train.csv")
dataset_train = dataset_train.dropna()
print(dataset_train["sentiment"].value_counts())
X = dataset_train.text.values
y = dataset_train.sentiment.values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

y_train = pd.Series(y_train)
y_test = pd.Series(y_test)

#dico = {"text":X, "sentiment":y}
#dfo = pd.DataFrame(data=dico)
#dfo.to_csv ("dataset_train.csv", index = False, header=True)

 
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
y_train = to_categorical(y_train, num_classes = 3)
y_test = to_categorical(y_test, num_classes = 3)
print(X_train)
print(y_train)

In [ ]:
vocab_size = 5000
embedding_dim = 16
max_length = 25
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
print(X_train)
print(type(X_train))
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

# Train
sequences_train = tokenizer.texts_to_sequences(X_train)
padded_train = pad_sequences(sequences_train, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(padded_train)

# Test
sequences_test = tokenizer.texts_to_sequences(X_test)
padded_test = pad_sequences(sequences_test, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
training_padded = np.array(padded_train)
training_label = np.array(y_train)
test_padded = np.array(padded_test)
test_label = np.array(y_test)

print(X[1312])
print(training_padded[0])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    #tf.keras.layers.LSTM(15, dropout=0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
num_epochs = 25
history = model.fit(training_padded, training_label, batch_size=16 ,epochs=num_epochs, validation_data=(test_padded, test_label), verbose=2)

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
with tpu_strategy.scope():
    mode5 = Sequential()
    mode5.add(tf.keras.layers.Embedding(vocab_size, 40, input_length=max_length))
    mode5.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    mode5.add(tf.keras.layers.Dense(512, activation='relu'))
    mode5.add(tf.keras.layers.Dropout(0.50))
    mode5.add(tf.keras.layers.Dense(3, activation='softmax'))
    mode5.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    history = mode5.fit(training_padded, training_label, epochs=20,validation_data=(test_padded, test_label))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(15, activation='relu'),
    tf.keras.layers.Dense(15, activation='relu'),
    #tf.keras.layers.LSTM(15, dropout=0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
num_epochs = 25
history = model.fit(training_padded, training_label, batch_size=32 ,epochs=num_epochs, validation_data=(test_padded, test_label), verbose=2)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
sentence = ["tek kelimeyle berbat hayal kırıklığına uğradım"]#, "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(sequences)
raw_prediction = model.predict(padded)
print(raw_prediction)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import nltk
import re
sentence = "sizinle tanışmak çok hoş"
description = nltk.word_tokenize(sentence)
metin = " ".join(description)
for i, word in enumerate(description):
    description[i] = lemmatizer(word,metin)
description = [str(word) for word in description if not word in stopwords.words()]
description = " ".join(description)
sequences = tokenizer.texts_to_sequences([description])
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

raw_prediction = model.predict(padded)
print(raw_prediction[0])